<a href="https://colab.research.google.com/github/SM24-Industrial-Software-Dev/ML-forecasting-NOx-levels/blob/ES-8-CBSA-Ingester/CBSA_Shapefile_Ingester.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Imports and Installations
from google.colab import auth
import google.auth
import ee
import requests
import zipfile
import io
import geemap

!pip install pycrs

  Preparing metadata (setup.py) ... done
  Created wheel for pycrs: filename=PyCRS-1.0.2-py3-none-any.whl size=32687 sha256=0ac2da979d7fe020f0913ba8560fef1350e5c765b91c51e1324524715497d0ec
  Stored in directory: /root/.cache/pip/wheels/47/1d/70/7a5bdf33347e7c75e95b06b1fa38f076a59a9506653cc24aff
Successfully built pycrs


In [ ]:
# Authenticate GEE
auth.authenticate_user()
credentials, _ = google.auth.default()
ee.Initialize(credentials, project='yu-summer-2024', opt_url='https://earthengine-highvolume.googleapis.com')

In [ ]:
# The method to retrieve a CBSA shapefile from the Census Bureau website and ingest it as a FeatureCollection
def ingest_cbsa_shapefile(year=2023, resolution='5m'):
  """
  Downloads a CBSA shapefile from the US Census Bureau, extracts it,
  and uploads it to Google Earth Engine (GEE).

  Args:
      year (int): The year of the shapefile (2023 by default, can range from 2014-2023)
      resolution (str): The resolution of the shapefile (5m by default, can be 500k, 5m, or 20m)

  Returns:
      The uploaded shapefile
  """
  # Validate the year and resolution inputs, setting to default if invalid
  if year < 2014:
      year = 2014
  elif year > 2023:
      year = 2023
  if resolution not in ['5m', '500k', '20m']:
      resolution = '5m'

  filename = f'cb_{year}_us_cbsa_{resolution}'
  # Download the shapefile
  response = requests.get(f'https://www2.census.gov/geo/tiger/GENZ{year}/shp/{filename}.zip')

  # Extract the shapefile
  with zipfile.ZipFile(io.BytesIO(response.content)) as zip_ref:
      zip_ref.extractall()

  # Upload the shapefile to GEE by reading it with Latin-1 encoding, which is commonly used for shapefiles
  return geemap.shp_to_ee(f'{filename}.shp', encoding='latin1')

# The actual retrieval
cbsa_geometries = ingest_cbsa_shapefile()